#### 1. Importing Data & Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Para encontrar a morada de cada
import reverse_geocoder as rg
from geopy.geocoders import Nominatim

#Para fazer mapas
import folium
from folium.plugins import HeatMap, MarkerCluster 

import Functions

In [2]:
customers = pd.read_csv('Customer Info Dataset.csv')

In [3]:
transactions = pd.read_csv('Customer Basket Dataset.csv')

In [4]:
prod_mapping = pd.read_excel('Product Mapping Excel File.xlsx')

## Customers Dataset

#### 2. Data Cleaning and Feature Extraction: ver duplicados, missing values, dtypes e por aí fora

 **Analisar Duplicados**

In [5]:
customers[customers['customer_id'].duplicated() == True]

,customer_id,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude


In [6]:
customers.set_index('customer_id', inplace=True)

In [7]:
customers[customers.duplicated() == True]

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,


 **Análise sumária e preliminar dos Dados**

In [8]:
customers.describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,30000.0,1.146700e+00,1.234111,0.000000e+00,0.000000,1.000000,1.000000,10.000000
teens_home,30000.0,5.129333e-01,0.912502,0.000000e+00,0.000000,0.000000,1.000000,9.000000
number_complaints,30000.0,7.643667e-01,0.836341,0.000000e+00,0.000000,1.000000,1.000000,9.000000
distinct_stores_visited,30000.0,6.120767e+00,6.511410,0.000000e+00,2.000000,3.000000,8.000000,21.000000
lifetime_spend_groceries,30000.0,3.978634e+03,5280.641466,0.000000e+00,220.000000,950.000000,5253.000000,18852.000000
lifetime_spend_electronics,30000.0,9.652388e+02,1755.323397,1.058527e+00,45.000000,194.000000,721.000000,6603.000000
typical_hour,30000.0,-inf,NaN,-inf,9.000000,14.000000,20.000000,23.000000
lifetime_spend_vegetables,30000.0,5.642078e+02,481.819086,1.000000e+00,247.000000,425.000000,785.000000,1568.000000
lifetime_spend_nonalcohol_drinks,30000.0,6.287501e+02,496.346400,0.000000e+00,244.000000,425.000000,949.000000,1671.000000
lifetime_spend_alcohol_drinks,30000.0,3.793445e+02,236.206400,0.000000e+00,193.000000,419.000000,537.000000,1048.000000


> Há aqui algos problemas: A variavel typical hour e lifetime spend videogames tem valores infinitos, o que dado o contexto das variaveis em si, condiciona a forma que poderão ser tratadas. 

In [9]:
customers[customers['lifetime_spend_videogames'] == -np.inf]

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,
1441,Fausto`s Supermarket,male,01/23/1989 11:26 AM,0.0,0.0,0.0,1.0,2.678482,1.095507,0.575646,...,2.613335,34425.0,1.556140,-inf,0.978006,0.199261,2008.0,961221.0,38.866242,-9.110871
3399,Sandy`s Supermarket,female,07/11/1970 12:57 PM,0.0,0.0,0.0,1.0,2.797932,1.141087,0.447940,...,2.623429,36152.0,1.555148,-inf,0.997246,0.201933,2009.0,NaN,38.866366,-9.111207
8086,Eugene`s Supermarket,male,02/29/1956 08:26 PM,0.0,0.0,0.0,1.0,2.802438,1.146242,0.173287,...,2.612483,34777.0,1.530623,-inf,0.951666,0.210895,2010.0,NaN,38.865887,-9.111353
9706,Michele`s Supermarket,female,12/06/1962 02:12 AM,0.0,0.0,0.0,1.0,2.844843,1.184050,0.708303,...,2.615854,35000.0,1.555148,-inf,0.997246,0.200211,2009.0,NaN,38.866273,-9.111174
17805,Robert`s Supermarket,male,09/22/1982 03:36 AM,0.0,0.0,0.0,1.0,2.751432,1.127715,0.599474,...,2.619223,35809.0,1.553152,-inf,1.006338,0.195712,2008.0,NaN,38.866607,-9.111206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689,Justin`s Supermarket,male,03/20/1959 03:59 PM,0.0,0.0,0.0,1.0,2.741558,1.113587,0.621227,...,2.613508,34834.0,1.552650,-inf,0.967800,0.203212,2011.0,NaN,38.866031,-9.111438
16270,Melissa`s Supermarket,female,05/11/1996 05:49 AM,0.0,0.0,0.0,1.0,2.768803,1.175120,0.519860,...,2.619885,34631.0,1.571034,-inf,0.987811,0.198526,2008.0,NaN,38.865962,-9.110773
10262,Francis`s Supermarket,male,11/05/1970 01:58 AM,0.0,0.0,0.0,1.0,2.788190,1.130447,0.693147,...,2.621949,34708.0,1.553152,-inf,0.957160,0.191255,2010.0,NaN,38.866438,-9.111502


In [10]:
customers[customers['typical_hour'] == -np.inf]

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,
8592,Charles`s Supermarket,male,01/18/1988 04:13 AM,0.0,0.0,0.0,1.0,2.64414,1.130447,-inf,...,2.615969,35005.0,1.540302,-inf,0.95716,0.196409,2013.0,NaN,38.866685,-9.111366
101,Annie`s Supermarket,female,11/23/1947 10:33 AM,0.0,0.0,0.0,1.0,2.65918,1.133150,-inf,...,2.608573,34702.0,1.553652,-inf,0.91589,0.194451,2009.0,910065.0,38.866048,-9.111718


Parece que são apenas clientes de supermercados com esta particularidade. Vamos então fazer um dataset separado para as pessoas dos supermercados  já que têm um behaviour tão diferente, e proceder a uma análise distinta dos mesmos depois. 

Por enquanto, vou apenas colocar os infinitos como 0 na variavel **lifetime_spend_videogames**, _pelo que o stor me deu a entender essa será uma boa approach_. Para as horas, ainda não sei o que devemos fazer.

**Dúvida para o prof: como deveremos interpretar a variavel _typical_hour_ neste contexto? E os valores das restantes variáveis**

In [11]:
customers['lifetime_spend_videogames'] = np.where(customers['lifetime_spend_videogames'] == -np.inf, 0, customers['lifetime_spend_videogames'])

In [12]:
(customers[customers['customer_name'].str.contains('Supermarket')]).describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,226.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
teens_home,226.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
number_complaints,226.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
distinct_stores_visited,226.0,1.000000e+00,0.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000
lifetime_spend_groceries,226.0,2.701444e+00,0.057132,2.423006e+00,2.669260,2.707140,2.741420,2.844843
lifetime_spend_electronics,226.0,1.149498e+00,0.026245,1.058527e+00,1.130447,1.151293,1.165860,1.222587
typical_hour,226.0,-inf,NaN,-inf,0.575646,0.693147,0.722593,0.783874
lifetime_spend_vegetables,226.0,2.124160e+00,0.026242,2.042546e+00,2.105281,2.126912,2.143535,2.183680
lifetime_spend_nonalcohol_drinks,226.0,2.302082e+00,0.012521,2.266444e+00,2.293678,2.303334,2.309999,2.334485
lifetime_spend_alcohol_drinks,226.0,2.704784e+00,0.002484,2.697901e+00,2.703110,2.704669,2.706443,2.712882


> Como podemos ver, os supermercados têm um comportamento totalmente distinto. Também eles tem _typical_hour_ sob o formato de infinitos (ou um formato super mau que não representa horas). 

 **Analisar missing values e corrigir dtypes;**

In [13]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 4239 to 2570
Data columns (total 23 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   customer_name                            30000 non-null  object 
 1   customer_gender                          30000 non-null  object 
 2   customer_birthdate                       30000 non-null  object 
 3   kids_home                                30000 non-null  float64
 4   teens_home                               30000 non-null  float64
 5   number_complaints                        30000 non-null  float64
 6   distinct_stores_visited                  30000 non-null  float64
 7   lifetime_spend_groceries                 30000 non-null  float64
 8   lifetime_spend_electronics               30000 non-null  float64
 9   typical_hour                             30000 non-null  float64
 10  lifetime_spend_vegetables                300

> Existem missing values somente na variável nr Loyalty card, que poderão se traduzir como "não-aplicável" (a pessoa não tem loyalty card). Não adianta imputar valores aqui, e sendo uma % tão grande de obs, também não ha grande utilidade em recorrer a esta var - mais vale fazer a variável binária. Agora vamos mudar os dtypes.

**converter o birthdate em datetime**

In [14]:
customers['customer_birthdate'] = pd.to_datetime(customers['customer_birthdate'])

In [15]:
customers

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,female,1999-09-10 15:11:00,0.0,0.0,0.0,2.0,91.0,20.0,22.0,...,53.0,48.0,47.0,105.0,85.0,0.192154,2017.0,NaN,38.754263,-9.163397
19330,Donald Robertson,male,1981-06-26 22:58:00,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,...,86.0,80.0,45.0,1964.0,85.0,0.098647,2015.0,NaN,38.777761,-9.156197
2272,Phd. Felicia Bradshaw,female,1987-01-31 09:08:00,1.0,1.0,0.0,20.0,397.0,20.0,9.0,...,204.0,170.0,46.0,5.0,408.0,0.497251,2008.0,NaN,38.785329,-9.127113
25288,Emma Williams,female,1942-06-17 00:02:00,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,...,1081.0,1068.0,617.0,1078.0,1956.0,0.206713,2010.0,NaN,38.744086,-9.156589
12829,Hilda Huerta,female,2000-01-31 11:43:00,0.0,0.0,0.0,2.0,82.0,22.0,21.0,...,39.0,39.0,48.0,103.0,83.0,0.200625,2015.0,NaN,38.756024,-9.162970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Phd. Karen Watts,female,1974-11-01 05:26:00,0.0,0.0,1.0,2.0,209.0,5474.0,20.0,...,21.0,11.0,57.0,1984.0,112.0,0.096366,2008.0,935162.0,38.734446,-9.171911
26447,Curtis Thompson,male,1986-02-21 11:27:00,0.0,0.0,0.0,3.0,185.0,1066.0,15.0,...,47.0,41.0,50.0,197.0,22.0,0.118998,2014.0,NaN,38.767070,-9.144301
182,Richard Carter,male,2001-01-14 05:49:00,0.0,0.0,0.0,2.0,83.0,23.0,23.0,...,55.0,52.0,51.0,94.0,77.0,0.199432,2016.0,953891.0,38.728487,-9.164140


In [16]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 4239 to 2570
Data columns (total 23 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customer_name                            30000 non-null  object        
 1   customer_gender                          30000 non-null  object        
 2   customer_birthdate                       30000 non-null  datetime64[ns]
 3   kids_home                                30000 non-null  float64       
 4   teens_home                               30000 non-null  float64       
 5   number_complaints                        30000 non-null  float64       
 6   distinct_stores_visited                  30000 non-null  float64       
 7   lifetime_spend_groceries                 30000 non-null  float64       
 8   lifetime_spend_electronics               30000 non-null  float64       
 9   typical_hour                         

#### **Separar clientes individuais de supermercados e Criar variáveis novas que poderão ser úteis**

Até agora, estas são as variáveis que temos:


_Para clientes individuais_

- **minors_household**: número de menores na household
- **Education**: educação da pessoa (pelo nome)
- **idade**
- **data de nascimento sem horas**: eu acho que isto poderá ser útil para promoções na altura do aniversário e tal :)

_Para supermercados_

_Para ambos_
- **address**: morada retirada com base 
- **total_lifetime_spent**: total gasto
- **loyalty_card**: var binária se tem ou nao loyalty card
- Será que fazemos % de valor gasto em cada tipo de categoria?

**loyalty card**

- será que podemos fazer este passo com a função do binário que está mais a baixo?

In [17]:
#loyalty_card
customers['loyalty_card'] = np.where(np.isnan(customers['loyalty_card_number']) == True, 0, 1)
customers.drop('loyalty_card_number', axis=1, inplace=True)

**total lifetime spend**

In [ ]:
#total_lifetime_spend
customers['total_lifetime_spend'] = customers[['lifetime_spend_vegetables',
       'lifetime_spend_nonalcohol_drinks', 'lifetime_spend_alcohol_drinks',
       'lifetime_spend_meat', 'lifetime_spend_fish', 'lifetime_spend_hygiene',
       'lifetime_spend_videogames']].sum(axis=1)

**location**

In [ ]:
customers['location'] = customers.apply(lambda row : Functions.get_address(row))

**separar supermarket e individuals**

In [18]:
supermarkets = customers[customers['customer_name'].str.contains('Supermarket')]

In [19]:
individuals = customers[~customers.isin(supermarkets)].dropna(how='all')

In [20]:
individuals.describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,29774.0,1.155404,1.234720,0.000000,0.000000,1.000000,1.000000,10.000000
teens_home,29774.0,0.516827,0.914860,0.000000,0.000000,0.000000,1.000000,9.000000
number_complaints,29774.0,0.770169,0.836844,0.000000,0.000000,1.000000,1.000000,9.000000
distinct_stores_visited,29774.0,6.159636,6.520716,0.000000,2.000000,3.000000,8.000000,21.000000
lifetime_spend_groceries,29774.0,4008.813629,5289.228508,0.000000,223.000000,961.000000,5267.000000,18852.000000
lifetime_spend_electronics,29774.0,972.556727,1759.954452,2.000000,46.000000,195.000000,734.750000,6603.000000
typical_hour,29774.0,14.461107,5.509949,0.000000,9.000000,14.000000,20.000000,23.000000
lifetime_spend_vegetables,29774.0,568.474340,481.139698,1.000000,252.000000,429.000000,787.000000,1568.000000
lifetime_spend_nonalcohol_drinks,29774.0,633.505206,495.205308,0.000000,250.000000,431.000000,952.000000,1671.000000
lifetime_spend_alcohol_drinks,29774.0,382.203433,234.802038,0.000000,196.000000,422.000000,537.000000,1048.000000


**minors in household**

In [21]:
individuals['minors_in_household'] = (individuals[['kids_home', 'teens_home']]).sum(axis=1)

### ATENÇÃO: FALTA PASSAR AS FUNÇÕES PARA O OUTRO DOCUMENTO DAS FUNÇÕES, DEIXEI AQUI PARA DAREM UMA VISTA DE OLHOS ANTES DE PASSARMOS

**processamento do birthdate**

In [22]:
def process_bithdate(df):
    #birthday
    df['birthday'] = df['customer_birthdate'].dt.day
    #birthmonth
    df['birthmonth'] = df['customer_birthdate'].dt.month
    #birthyear
    df['birthyear'] = df['customer_birthdate'].dt.year

    return df

In [23]:
individuals = Functions.process_bithdate(individuals)

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,percentage_of_products_bought_promotion,year_first_transaction,latitude,longitude,loyalty_card,total_lifetime_spend,minors_in_household,birthday,birthmonth,birthyear
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,female,1999-09-10 15:11:00,0.0,0.0,0.0,2.0,91.0,20.0,22.0,...,0.192154,2017.0,38.754263,-9.163397,0.0,1434.0,0.0,10,9,1999
19330,Donald Robertson,male,1981-06-26 22:58:00,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,...,0.098647,2015.0,38.777761,-9.156197,0.0,4064.0,1.0,26,6,1981
2272,Phd. Felicia Bradshaw,female,1987-01-31 09:08:00,1.0,1.0,0.0,20.0,397.0,20.0,9.0,...,0.497251,2008.0,38.785329,-9.127113,0.0,1336.0,2.0,31,1,1987
25288,Emma Williams,female,1942-06-17 00:02:00,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,...,0.206713,2010.0,38.744086,-9.156589,0.0,5676.0,7.0,17,6,1942
12829,Hilda Huerta,female,2000-01-31 11:43:00,0.0,0.0,0.0,2.0,82.0,22.0,21.0,...,0.200625,2015.0,38.756024,-9.162970,0.0,1266.0,0.0,31,1,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Phd. Karen Watts,female,1974-11-01 05:26:00,0.0,0.0,1.0,2.0,209.0,5474.0,20.0,...,0.096366,2008.0,38.734446,-9.171911,1.0,4047.0,0.0,1,11,1974
26447,Curtis Thompson,male,1986-02-21 11:27:00,0.0,0.0,0.0,3.0,185.0,1066.0,15.0,...,0.118998,2014.0,38.767070,-9.144301,0.0,1165.0,0.0,21,2,1986
182,Richard Carter,male,2001-01-14 05:49:00,0.0,0.0,0.0,2.0,83.0,23.0,23.0,...,0.199432,2016.0,38.728487,-9.164140,1.0,1385.0,0.0,14,1,2001


**education**

In [24]:
#extrair educação
individuals['education'] = individuals['customer_name'].apply(lambda row : Functions.extract_education(row))

In [25]:
individuals['education'].unique()

array(['Basic', 'Doctoral Degree', 'Bachelor Degree', 'Masters Degree'],
      dtype=object)

**encoding das não binárias: education, (...)**

In [26]:
# função que faz o encoding não binário
def categorical_encoding(df, col_name, replace_dict):
    new_col_name = col_name + '_encoded'
    df[new_col_name] = df[col_name].map(replace_dict)
    # df = df.drop(col_name, axis=1) # faz o drop da coluna inicial
    return df

In [27]:
education_map = {
    'Basic': 0,
    'Bachelor Degree': 1,
    'Masters Degree': 2,
    'Doctoral Degree': 3
}
Functions.categorical_encoding(individuals, 'education', education_map)

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,latitude,longitude,loyalty_card,total_lifetime_spend,minors_in_household,birthday,birthmonth,birthyear,education,education_encoded
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,female,1999-09-10 15:11:00,0.0,0.0,0.0,2.0,91.0,20.0,22.0,...,38.754263,-9.163397,0.0,1434.0,0.0,10,9,1999,Basic,0
19330,Donald Robertson,male,1981-06-26 22:58:00,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,...,38.777761,-9.156197,0.0,4064.0,1.0,26,6,1981,Basic,0
2272,Phd. Felicia Bradshaw,female,1987-01-31 09:08:00,1.0,1.0,0.0,20.0,397.0,20.0,9.0,...,38.785329,-9.127113,0.0,1336.0,2.0,31,1,1987,Doctoral Degree,3
25288,Emma Williams,female,1942-06-17 00:02:00,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,...,38.744086,-9.156589,0.0,5676.0,7.0,17,6,1942,Basic,0
12829,Hilda Huerta,female,2000-01-31 11:43:00,0.0,0.0,0.0,2.0,82.0,22.0,21.0,...,38.756024,-9.162970,0.0,1266.0,0.0,31,1,2000,Basic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Phd. Karen Watts,female,1974-11-01 05:26:00,0.0,0.0,1.0,2.0,209.0,5474.0,20.0,...,38.734446,-9.171911,1.0,4047.0,0.0,1,11,1974,Doctoral Degree,3
26447,Curtis Thompson,male,1986-02-21 11:27:00,0.0,0.0,0.0,3.0,185.0,1066.0,15.0,...,38.767070,-9.144301,0.0,1165.0,0.0,21,2,1986,Basic,0
182,Richard Carter,male,2001-01-14 05:49:00,0.0,0.0,0.0,2.0,83.0,23.0,23.0,...,38.728487,-9.164140,1.0,1385.0,0.0,14,1,2001,Basic,0


**encoding das binárias: gender, (...)**

In [28]:
# função que faz o encoding binário
def binary_encoding(df, col_name, replace_dict):
    df[col_name] = df[col_name].replace(replace_dict)
    return df

In [29]:
gender_dict = {'male': 0,
               'female': 1} 
binary_encoding(individuals, 'customer_gender', gender_dict)

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,latitude,longitude,loyalty_card,total_lifetime_spend,minors_in_household,birthday,birthmonth,birthyear,education,education_encoded
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,1,1999-09-10 15:11:00,0.0,0.0,0.0,2.0,91.0,20.0,22.0,...,38.754263,-9.163397,0.0,1434.0,0.0,10,9,1999,Basic,0
19330,Donald Robertson,0,1981-06-26 22:58:00,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,...,38.777761,-9.156197,0.0,4064.0,1.0,26,6,1981,Basic,0
2272,Phd. Felicia Bradshaw,1,1987-01-31 09:08:00,1.0,1.0,0.0,20.0,397.0,20.0,9.0,...,38.785329,-9.127113,0.0,1336.0,2.0,31,1,1987,Doctoral Degree,3
25288,Emma Williams,1,1942-06-17 00:02:00,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,...,38.744086,-9.156589,0.0,5676.0,7.0,17,6,1942,Basic,0
12829,Hilda Huerta,1,2000-01-31 11:43:00,0.0,0.0,0.0,2.0,82.0,22.0,21.0,...,38.756024,-9.162970,0.0,1266.0,0.0,31,1,2000,Basic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Phd. Karen Watts,1,1974-11-01 05:26:00,0.0,0.0,1.0,2.0,209.0,5474.0,20.0,...,38.734446,-9.171911,1.0,4047.0,0.0,1,11,1974,Doctoral Degree,3
26447,Curtis Thompson,0,1986-02-21 11:27:00,0.0,0.0,0.0,3.0,185.0,1066.0,15.0,...,38.767070,-9.144301,0.0,1165.0,0.0,21,2,1986,Basic,0
182,Richard Carter,0,2001-01-14 05:49:00,0.0,0.0,0.0,2.0,83.0,23.0,23.0,...,38.728487,-9.164140,1.0,1385.0,0.0,14,1,2001,Basic,0


In [30]:
#Limpar o nome (tirar a educação)
individuals['customer_name'] = individuals['customer_name'].apply(lambda row: Functions.clean_names(row))

In [31]:
individuals

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,latitude,longitude,loyalty_card,total_lifetime_spend,minors_in_household,birthday,birthmonth,birthyear,education,education_encoded
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,1,1999-09-10 15:11:00,0.0,0.0,0.0,2.0,91.0,20.0,22.0,...,38.754263,-9.163397,0.0,1434.0,0.0,10,9,1999,Basic,0
19330,Donald Robertson,0,1981-06-26 22:58:00,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,...,38.777761,-9.156197,0.0,4064.0,1.0,26,6,1981,Basic,0
2272,Felicia Bradshaw,1,1987-01-31 09:08:00,1.0,1.0,0.0,20.0,397.0,20.0,9.0,...,38.785329,-9.127113,0.0,1336.0,2.0,31,1,1987,Doctoral Degree,3
25288,Emma Williams,1,1942-06-17 00:02:00,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,...,38.744086,-9.156589,0.0,5676.0,7.0,17,6,1942,Basic,0
12829,Hilda Huerta,1,2000-01-31 11:43:00,0.0,0.0,0.0,2.0,82.0,22.0,21.0,...,38.756024,-9.162970,0.0,1266.0,0.0,31,1,2000,Basic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Karen Watts,1,1974-11-01 05:26:00,0.0,0.0,1.0,2.0,209.0,5474.0,20.0,...,38.734446,-9.171911,1.0,4047.0,0.0,1,11,1974,Doctoral Degree,3
26447,Curtis Thompson,0,1986-02-21 11:27:00,0.0,0.0,0.0,3.0,185.0,1066.0,15.0,...,38.767070,-9.144301,0.0,1165.0,0.0,21,2,1986,Basic,0
182,Richard Carter,0,2001-01-14 05:49:00,0.0,0.0,0.0,2.0,83.0,23.0,23.0,...,38.728487,-9.164140,1.0,1385.0,0.0,14,1,2001,Basic,0


In [32]:
individuals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29774 entries, 4239 to 2570
Data columns (total 30 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customer_name                            29774 non-null  object        
 1   customer_gender                          29774 non-null  int64         
 2   customer_birthdate                       29774 non-null  datetime64[ns]
 3   kids_home                                29774 non-null  float64       
 4   teens_home                               29774 non-null  float64       
 5   number_complaints                        29774 non-null  float64       
 6   distinct_stores_visited                  29774 non-null  float64       
 7   lifetime_spend_groceries                 29774 non-null  float64       
 8   lifetime_spend_electronics               29774 non-null  float64       
 9   typical_hour                         

**converter types das variaveis**

In [33]:
def integer_convert(df, cols):
    # Converte as colunas especificadas em cols de float para int
    for col in cols:
        df[col] = df[col].astype(int)
    return df


In [34]:
colums_to_int = individuals[['number_complaints', 'distinct_stores_visited', 
                             'kids_home', 'teens_home', 'year_first_transaction', 
                             'typical_hour', 'lifetime_total_distinct_products', 'loyalty_card', 
                             'minors_in_household']]

integer_convert(individuals, colums_to_int)

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,latitude,longitude,loyalty_card,total_lifetime_spend,minors_in_household,birthday,birthmonth,birthyear,education,education_encoded
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,1,1999-09-10 15:11:00,0,0,0,2,91.0,20.0,22,...,38.754263,-9.163397,0,1434.0,0,10,9,1999,Basic,0
19330,Donald Robertson,0,1981-06-26 22:58:00,1,0,0,2,197.0,5958.0,21,...,38.777761,-9.156197,0,4064.0,1,26,6,1981,Basic,0
2272,Felicia Bradshaw,1,1987-01-31 09:08:00,1,1,0,20,397.0,20.0,9,...,38.785329,-9.127113,0,1336.0,2,31,1,1987,Doctoral Degree,3
25288,Emma Williams,1,1942-06-17 00:02:00,3,4,1,2,5282.0,237.0,12,...,38.744086,-9.156589,0,5676.0,7,17,6,1942,Basic,0
12829,Hilda Huerta,1,2000-01-31 11:43:00,0,0,0,2,82.0,22.0,21,...,38.756024,-9.162970,0,1266.0,0,31,1,2000,Basic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Karen Watts,1,1974-11-01 05:26:00,0,0,1,2,209.0,5474.0,20,...,38.734446,-9.171911,1,4047.0,0,1,11,1974,Doctoral Degree,3
26447,Curtis Thompson,0,1986-02-21 11:27:00,0,0,0,3,185.0,1066.0,15,...,38.767070,-9.144301,0,1165.0,0,21,2,1986,Basic,0
182,Richard Carter,0,2001-01-14 05:49:00,0,0,0,2,83.0,23.0,23,...,38.728487,-9.164140,1,1385.0,0,14,1,2001,Basic,0


In [35]:
individuals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29774 entries, 4239 to 2570
Data columns (total 30 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customer_name                            29774 non-null  object        
 1   customer_gender                          29774 non-null  int64         
 2   customer_birthdate                       29774 non-null  datetime64[ns]
 3   kids_home                                29774 non-null  int64         
 4   teens_home                               29774 non-null  int64         
 5   number_complaints                        29774 non-null  int64         
 6   distinct_stores_visited                  29774 non-null  int64         
 7   lifetime_spend_groceries                 29774 non-null  float64       
 8   lifetime_spend_electronics               29774 non-null  float64       
 9   typical_hour                         

#### 3. Univariate Analysis: Vamos plottar os gráficos de cada variavel separadamente; analisar distribuições e possíveis outliers

Em primeiro lugar separo variaveis: discretas, continuas e categóricas (incluido as binárias). Não sei sinceramente se isto vale a pena ou se está bem feito, mas era só para facilitar os plots.

In [ ]:
categorical = ['loyalty_card', 'customer_name', 'education', 'gender', 'address']
discrete = ['kids_home', 'teens_home',
       'number_complaints', 'distinct_stores_visited']
continuous = ['lifetime_spend_vegetables',
       'lifetime_spend_nonalcohol_drinks', 'lifetime_spend_alcohol_drinks',
       'lifetime_spend_meat', 'lifetime_spend_fish', 'lifetime_spend_hygiene',
       'lifetime_spend_videogames', 'total_lifetime_spent', 'percentage_of_products_bought_promotion']

> **distribuição de observações por localização** (Mapa)

In [ ]:
map_lisbon = folium.Map(location=[38.7223, -9.1393], zoom_start=12)
marker_cluster = MarkerCluster().add_to(map_lisbon)
for index, row in customers.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(marker_cluster)

# Display the map
map_lisbon

#### 4. Multivariate Analysis: Correlação e testes de independencia, vamos também ver se existe alguma relação entre variaveis :)

## Basket Dataset